In [1]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/leonardo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import pandas as pd

df = pd.read_csv('../dados/acordaos-selecionada-lematizados.csv', sep = '|', encoding = 'utf-8')
df.head()

,acordao,areas,texto_lematizado
0,297/2016-P,Responsabilidade,tribunal de conta de o união tc XXNUMERACAO\r\...
1,366/2016-P,Finanças Públicas,tribunal de conta de o união tc XXNUMERACAO\r\...
2,944/2016-P,Responsabilidade,tribunal de conta de o união tc XXNUMERACAO\r\...
3,30/2016-P,Direito Processual,tribunal de conta de o união tc XXNUMERACAO\r\...
4,55/2016-P,Pessoal,; - ; ; wania lucia pasquarelli de o nasciment...


In [3]:
df.shape

(10524, 3)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 

textos = df['texto_lematizado'].values
vetorizador = TfidfVectorizer(stop_words = stopwords.words('portuguese'))  #token_pattern=r'\S+', 
vetorizador.fit(textos)
print(len(vetorizador.vocabulary_), 'termos distintos')

151719 termos distintos


In [5]:
matriz = vetorizador.transform(textos)
matriz[0]

<1x151719 sparse matrix of type '<class 'numpy.float64'>'
	with 607 stored elements in Compressed Sparse Row format>

In [6]:
def extrair_tfidf(documento):
    indices = []
    acordao = []
    tf_idf = []
    for termo in textos[documento].split():
        if termo in vetorizador.vocabulary_:
            indice = vetorizador.vocabulary_[termo]
            indices.append(indice)
            acordao.append(termo)
            tf_idf.append(matriz[documento,indice])
    return acordao, indices, tf_idf

In [9]:
def extrair_tamanho(documento):
    qtde = 0
    for termo in textos[documento].split():
        if termo in vetorizador.vocabulary_:
            qtde += 1
    return qtde
extrair_tamanho(9000)

4225

In [10]:
for doc in range(0, len(textos)):
    df.loc[doc, 'tamanho'] = extrair_tamanho(doc)
df['tamanho'].describe()

count     10524.000000
mean       5215.540954
std       10217.056045
min         283.000000
25%        1847.000000
50%        3234.000000
75%        5989.500000
max      604173.000000
Name: tamanho, dtype: float64

In [11]:
import numpy as np

np.percentile(df['tamanho'], 90), np.percentile(df['tamanho'], 95), np.percentile(df['tamanho'], 99)

(10753.7, 15246.80000000001, 29011.620000000046)

In [12]:
def filtrar_texto(acordao, tf_idf, maximo):
    derivado = ''
    original = len(tf_idf)
    if maximo >= original:
        minimo = 0
    else:
        minimo = np.percentile(tf_idf,100-100*maximo/original)
    for indice, valor in enumerate(tf_idf):
        if valor > minimo:
            derivado += acordao[indice] + ' '
    return derivado[:-1]
filtrar_texto(acordao, tf_idf, 1000)

'órgão agência nacional transporte terrestre giovani machado gonçalves agência nacional transporte terrestre agência nacional transporte terrestre agência nacional transporte terrestre giovani machado gonçalves servir antt ceder tce desconto valor antt ser giovani machado gonçalves agência nacional transporte terrestre antt giovani machado gonçalves servir giovani machado gonçalves valor mpog antt período ceder goiás agência nacional transporte terrestre servir giovani machado gonçalves período ceder goiás agência nacional transporte terrestre goiás giovani machado gonçalves agência nacional transporte terrestre antt giovani machado gonçalves agência nacional transporte terrestre antt desconto contracheques servir giovani machado gonçalves desconto valor ser reembolsar antt goiás tce período servir giovani machado gonçalves ceder agência órgão valor descontar antt contracheques servir giovani machado gonçalves tce período ceder agência órgão ser antt tce valor reembolsar agência períod

In [13]:
filtrar_texto(acordao, tf_idf, 8000)

'tribunal conta união tc grupo classe tc natureza representação pedir reexame órgão entidade agência nacional transporte terrestre responsável giovani machado gonçalves interessar agência nacional transporte terrestre representação legal renata amar ferreira outro representar agência nacional transporte terrestre agência nacional transporte terrestre victor aguiar jardim amorim oab df outro representar giovani machado gonçalves sumário representação percepção indevido remuneração servir antt ceder tce determinação desconto valor perceber indevido âmbito processo administrativo disciplinar instaurar antt incompetência tcu apreciar legitimidade ato praticar âmbito processo administrativo disciplinar incompetência tcu tutela interesse privado ausência interesse recursal conhecer relatório adotar relatório manifestação unidade técnico cujo termo ser seguinte di histórico processual tratar pedido reexame interpor senhor giovani machado gonçalves peça através insurgir contra acórdão peça mod

In [14]:
for doc in range(0, len(textos)):
    acordao, _ , tf_idf = extrair_tfidf(doc)
    df.loc[doc, 'filtrado_6000'] = filtrar_texto(acordao, tf_idf, 6000)
    df.loc[doc, 'filtrado_500']  = filtrar_texto(acordao, tf_idf, 500)
    if (doc + 1) % 1000 == 0:
        print('doc', doc+1, '...')

doc 1000 ...
doc 2000 ...
doc 3000 ...
doc 4000 ...
doc 5000 ...
doc 6000 ...
doc 7000 ...
doc 8000 ...
doc 9000 ...
doc 10000 ...


In [15]:
df.head(10)

,acordao,areas,texto_lematizado,tamanho,filtrado_6000,filtrado_500
0,297/2016-P,Responsabilidade,tribunal de conta de o união tc XXNUMERACAO\r\...,1704.0,tribunal conta união tc grupo classe plenário ...,conta conta gerência instituto nacional seguro...
1,366/2016-P,Finanças Públicas,tribunal de conta de o união tc XXNUMERACAO\r\...,3330.0,tribunal conta união tc grupo classe ii plenár...,senado petróleo petróleo regime aduaneiro expo...
2,944/2016-P,Responsabilidade,tribunal de conta de o união tc XXNUMERACAO\r\...,842.0,tribunal conta união tc grupo classe plenário ...,tribunal conta plenário embargo declaração aco...
3,30/2016-P,Direito Processual,tribunal de conta de o união tc XXNUMERACAO\r\...,1379.0,tribunal conta união tc grupo classe plenário ...,embargo declaração inss recorrente marco antôn...
4,55/2016-P,Pessoal,; - ; ; wania lucia pasquarelli de o nasciment...,5887.0,wania lucia pasquarelli nascimentotcuwania luc...,senac senac senac empresa senac giselle araújo...
5,27/2016-P,Licitação,tribunal de conta de o união tc XXNUMERACAO\r\...,2025.0,tribunal conta união tc grupo classe plenário ...,apenso recurso natal nilton pascoal cpf walden...
6,38/2016-P,Pessoal,tribunal de conta de o união tc XXNUMERACAO\r\...,3292.0,tribunal conta união tc grupo classe plenário ...,aposentadoria sebastião aposentadoria gratific...
7,113/2016-P,Licitação,tribunal de conta de o união tc XXNUMERACAO\r\...,3425.0,tribunal conta união tc grupo classe plenário ...,dixtal biomédico pró life equipar eireli dixta...
8,119/2016-P,Licitação,tribunal de conta de o união tc XXNUMERACAO\r\...,5943.0,tribunal conta união tc grupo classe plenário ...,snake empresa pregão pregão trt vigilância emp...
9,412/2016-1,Pessoal,tribunal de conta de o união tc XXNUMERACAO\r\...,3849.0,tribunal conta união tc grupo classe tc nature...,aposentadoria enoe pimenta basso aposentadoria...


In [16]:
df.to_csv('../dados/acordaos-selecionada-filtrados-500.csv', sep = '|', columns = ['acordao', 'areas', 'filtrado_500'], index=False)

In [17]:
df.to_csv('../dados/acordaos-selecionada-filtrados-6000.csv', sep = '|', columns = ['acordao', 'areas', 'filtrado_6000'], index=False)